*Note: This script is an effort to replicate the results from the paper "Talk of the Network: A Complex Systems Look at the Underlying Process of Word-of-Mouth", Goldenberg, Libai and Muller (2001). This is a self-didactic attempt.*

In [ ]:
using Distributions

In [ ]:
srand(20130810)

# 1. Introduction 

This paper explores the pattern of personal communication betwee an individual's core friends group (strong ties) and a wider set of acquaintances (weak ties). This remarkable study is one of the first ones in marketing that explored the influence of social networks on the diffusion of marketing messages. The key questions investogated in the context of information dissemination are:

- What matters more - strong ties or weak ties?
- What effect does the size of an average individuals network have?
- How does advertising interact with the diffusion through weak ties and that through strong ties

# 2. Building the network substrates

## 2.1 The node object

Since this study employs a set of synthetic networks, where each of the nodes have a fixed number of strong ties ($s$) and weak ties ($w$), we cannot use existing graph types to build these networks. However, given the simple configuration of the network, we conceptualize a network as a list of node objects. Each node object has the id, a vector of strong ties, a vector of weak ties and activation status as the parameters of the node. 

In [ ]:
mutable struct Node
    id::Int
    weak_ties::Vector{Int} 
    strong_ties::Vector{Int} 
    status::Bool
    activation_prob::Float64
end

## 2.2 Initializing the network

We initialize the network as a list of empty `Node` objects and then build the neighborhoods of individual nodes by adhering to the number of strong and weak ties that each `Node` has.

In [ ]:
function initialize_network(n_nodes::Int, n_strong_ties::Int, n_weak_ties::Int)
    
    # Initialize an empty network
    
    G = [Node(i, [], [], false, 0.0) for i in 1:n_nodes] 
    node_ids = [node.id for node in G]
    
    # Wire the network according to the number of strong and weak ties
    # When wiring with random nodes, take care that the subject node and
    # already existing neighbors are not sampled again
    
    for node in G
        while length(node.weak_ties) < n_weak_ties
            rand_nbr = sample(node_ids[1:end .!= node.id])
            if !(rand_nbr in node.weak_ties || rand_nbr in node.strong_ties)
                push!(node.weak_ties, rand_nbr)
            end
        end
        while length(node.strong_ties) < n_strong_ties
            rand_nbr = sample(node_ids[1:end .!= node.id])
            if !(rand_nbr in node.weak_ties || rand_nbr in node.strong_ties)
                push!(node.strong_ties, rand_nbr)
            end
        end
    end
    
    return G
end

# 3. Model

## 3.1 Assumptions

Each individual in the substrate network (referred to as nodes) are connected to the same number of strong ties (varied from 5 - 29) and weak ties (varied from 5 - 29). The probability of activation of a node, i.e., an uninformed individual turning to informed can happen in three ways: through a strong tie with probability $\beta_s$, through a weak tie with probability $\beta_w$ or through external marketing efforts with probability $\alpha$. In line with conventional wisdom, we assume $\alpha < \beta_w < \beta_s$. 

At timestep $t$, if an individual is connected to $m$ strong ties and $j$ weak ties, the probability of the individual being informed in this time step is:

$$
p(t) = 1 - (1- \alpha)(1 - \beta_w)^j(1 - \beta_s)^m
$$

We are interested in two outcome variables:
1. The number of time steps elapsed till 15% of the network engages 
2. The number of time steps elapsed till 95% of the network engages

## 3.3 Execution

- At $t = 0$, the status of all nodes is set to `false`

- For each node, the probability of being informed is calculated as per the above equation. A random draw $U$ is made from a standard uniform distribution and compared with the probability. If $U < p(t)$ the status of the node is changed to `true`

- In each successive time step the previous step is repeated till 95% of the total network (of size 3000) engages

We now look at several helper functions that execute the above logic

### 3.3.1 Reset node status

At the beginning of each simulation, we call the following function to set the status of all the nodes to `false`. This is required if we wish to run more than one simulation at each point in the parameter space.

In [ ]:
function reset_node_status!(G::Vector{Node})
    for node in G
        node.status = false
    end
    
    return nothing
end

### 3.3.2 Activation probability

At each time step, the vector holding the probabilty of activation for each node is calculated using the following function. We count the number of activated strong and weak ties for each node and use the above formula to compute the activation probability.

In [ ]:
function update_activation_prob!(G::Vector{Node}, alpha::Float64, beta_w::Float64, beta_s::Float64)
    
    for node in G

        n_active_weak_ties, n_active_strong_ties = 0, 0
        
        weak_ties = G[node.weak_ties]
        strong_ties = G[node.strong_ties]

        for weak_tie in weak_ties
            if weak_tie.status == true
                n_active_weak_ties += 1
            end
        end

        for strong_tie in strong_ties
            if strong_tie.status == true
                n_active_strong_ties += 1
            end
        end

        node.activation_prob = 1 - (1 - alpha) * (1 - beta_w)^n_active_weak_ties * (1 - beta_s)^n_active_strong_ties

    end 
    
    return nothing
end

### 3.3.3 Update node status

At each time step the status of all the nodes is updated according to the calculated probability of activation. 

In [ ]:
function update_status!(G::Vector{Node}, alpha::Float64, beta_w::Float64, beta_s::Float64)
    for node in G
        update_activation_prob!(G, alpha, beta_w, beta_s)
        
        if rand(Uniform()) < node.activation_prob
            node.status = true
        end
    end
    
    return nothing
end

### 3.3.4 Simulation on the parameter space

The function `execute_simulation` puts together the scaffolding to set up the parameter space $(s, w, \alpha, \beta_w, \beta_s)$ and execute diffusion along the network. From what I can gather from the paper, one simulation was carried out at each point on the parameter space. No further details regarding the execution are mentioned except that since each parameter has 7 levels, so in a factorial design a total of $7^5 = 16,808$ simulations were executed.

Also, I am assuming that the network is drawn at random for each run of the simulation.

In [ ]:
println("Number of strong ties per node (s): ", floor.(Int, linspace(5, 29, 7)))
println("Number of weak ties per node(w): ", floor.(Int, linspace(5, 29, 7)))
println("Effect of advertising (α): ", collect(linspace(0.0005, 0.01, 7)))
println("Effect of weak ties (β_w): ", collect(linspace(0.005, 0.015, 7)))
println("Effect of strong ties (β_s): ", collect(linspace(0.01, 0.07, 7)))

In [ ]:
parameter_space = [(s, w, alpha, beta_w, beta_s) for s in floor.(Int, linspace(5, 29, 7)), 
                                                     w in floor.(Int, linspace(5, 29, 7)),
                                                     alpha in linspace(0.0005, 0.01, 7),
                                                     beta_w in linspace(0.005, 0.015, 7),
                                                     beta_s in linspace(0.01, 0.07, 7)]

size(parameter_space), length(parameter_space)

In [ ]:
function execute_simulation(parameter_space, n_nodes::Int, T::Int)
    
    # n_nodes dictates how big the network will be
    # T dictates the number of time steps for the simulation    
    
    output = Matrix{Any}(length(parameter_space) * T, 7)
    colnames = ["s", "w", "alpha", "beta_w", "beta_s", "t", "num_engaged"]
    i = 1 # Index the output
    
    # Rewiring the network each time is expensive. We can cut down repeats of the same rewiring process
    # by building the network only when the parameters used to build the network have changed.
    
    old_s, old_w = parameter_space[1][1:2]
    G = initialize_network(n_nodes, old_s, old_w)
    
    for (s, w, alpha, beta_w, beta_s) in parameter_space[1:end]
        
        # Rewire the network only if the network creation parameters have changed
        # Once initialize_network is fast, this portion of the code can be removed
        
        if !(old_s == s && old_w == w)
            G = initialize_network(n_nodes, s, w)
        end
        reset_node_status!(G)
        
        println("Beginning simulation on setting $((s, w, alpha, beta_w, beta_s)) at : ", Dates.format(now(), "HH:MM"))
        for t in 1:T
            update_status!(G, alpha, beta_w, beta_s)
            num_engaged = sum([node.status for node in G])
            output[i, :] = [s, w, alpha, beta_w, beta_s, t, num_engaged]
            i += 1
        end
    
        old_s, old_w = s, w
    end
    
    return [colnames; output]
end

In [ ]:
@time results = execute_simulation(parameter_space, 3000, 10)